In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [2]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

57344/57026 [==============================] - 0s 1us/step


In [3]:
print (train_data.shape, train_targets.shape)

(404, 13) (404,)


In [4]:
print (train_data[0], train_targets[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ] 15.2


In [5]:
print (train_targets[0])

15.2


여기서 볼 수 있듯이 404개의 훈련 샘플과 102개의 테스트 샘플이 있고 모두 13개의 수치 특성을 가지고 있습니다. 13개의 특성은 다음과 같습니다:

1. Per capita crime rate.
2. Proportion of residential land zoned for lots over 25,000 square feet.
3. Proportion of non-retail business acres per town.
4. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
5. Nitric oxides concentration (parts per 10 million).
6. Average number of rooms per dwelling.
7. Proportion of owner-occupied units built prior to 1940.
8. Weighted distances to five Boston employment centres.
9. Index of accessibility to radial highways.
10. Full-value property-tax rate per $10,000.
11. Pupil-teacher ratio by town.
12. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
13. % lower status of the population.

타깃은 주택의 중간 가격으로 천달러 단위입니다:

Data Normalization 

In [0]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [0]:
from keras import models
from keras import layers

def build_model():
    # 동일한 모델을 여러 번 생성할 것이므로 함수를 만들어 사용합니다
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

마지막 레어에는 Activation function이 없습니다. 회귀 문제(어떤 값을 예측하는 문제)를 풀때 이와 같이 구성합니다.
평균 제곱 오차 (mean squared error)
절대 평균 오차 (mean absolute error)

0.5 indicate $500

**K-Fold Validation**

In [8]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 10
all_scores = []
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]
        ], axis=0)

    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=10, verbose=1)
    
    # 검증 세트로 모델 평가
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

처리중인 폴드 # 0
Epoch 1/10
303/303 [==============================] - 0s 952us/step - loss: 560.9824 - mae: 22.0019
Epoch 2/10
303/303 [==============================] - 0s 109us/step - loss: 477.1412 - mae: 19.9768
Epoch 3/10
303/303 [==============================] - 0s 108us/step - loss: 357.3524 - mae: 16.9537
Epoch 4/10
303/303 [==============================] - 0s 117us/step - loss: 215.2467 - mae: 12.5464
Epoch 5/10
303/303 [==============================] - 0s 115us/step - loss: 101.5767 - mae: 7.9021
Epoch 6/10
303/303 [==============================] - 0s 110us/step - loss: 55.7236 - mae: 5.5835
Epoch 7/10
303/303 [==============================] - 0s 110us/step - loss: 38.6777 - mae: 4.5231
Epoch 8/10
303/303 [==============================] - 0s 118us/step - loss: 31.3709 - mae: 3.9733
Epoch 9/10
303/303 [==============================] - 0s 119us/step - loss: 27.7901 - mae: 3.6739
Epoch 10/10
303/303 [==============================] - 0s 114us/step - loss: 25.6583 - mae: 3.538

In [9]:
all_scores

[3.3137576580047607, 3.2725017070770264, 2.95336651802063, 3.7795259952545166]

In [10]:
np.mean(all_scores)

3.3297879695892334

In [0]:
# 아래 모델은 k fold cross validation에서 마지막에 훈련된 모델입니다.
# 본 실습에서 별도의 test data set을 구성하지 않았기 때문에, vadliation data set을 이용해 house price 를 예측prediction 해보겠습니다.

predictedHousePrice = model.predict(val_data)

In [12]:
# validation set 은 100개의 데이터 셋으로 구성되어 있습니다만 아래와 같이 1대1 비교를 해보겠습니다.
# val_data[0] 에 대한 예측값 predictedHousePrice[0]
# val_data[0] 에 대한 실제값 val_targets[0]
print (predictedHousePrice[0], val_targets[0])

[27.53785] 36.2


In [0]:
new_data = [1.47, 0.0, 8.14, 0.0, 0.53, 6.1, 91.7, 4.9769,  4.12,  307.6 ,  22,  296.9, 18.72]

In [14]:
new_data

[1.47, 0.0, 8.14, 0.0, 0.53, 6.1, 91.7, 4.9769, 4.12, 307.6, 22, 296.9, 18.72]

In [0]:
new_data -= mean
new_data /= std

In [16]:
print (new_data.shape)

(13,)


In [0]:
new_data = np.asarray(new_data)

In [0]:
new_data = np.expand_dims(new_data, axis=0)

In [19]:
print (new_data.shape)

(1, 13)


In [20]:
model.predict(new_data)

array([[14.348082]], dtype=float32)